In [11]:
pip install dotenv pypdf langchain langchain-community docx2txt -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

True

In [14]:
pip install wikipedia -q 


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [16]:
def load_document(file):
    import os

    name, extension = os.path.splitext(file)

    if extension == ".pdf":
        from langchain_classic.document_loaders import PyPDFLoader

        print(f"Loading {file}")
        loader = PyPDFLoader(file)
    elif extension == ".docx":
        from langchain_classic.document_loaders import Docx2txtLoader

        loader = Docx2txtLoader(file)
    else:
        print("Document format is not supported")
        return None

    data = loader.load()
    return data


# wikipedia


def load_from_wikipedia(query, lang="en", load_max_docs=2):
    from langchain_classic.document_loaders import WikipediaLoader

    loader = WikipediaLoader(query=query, lang=lang, load_max_docs=load_max_docs)
    data = loader.load()
    return data

## Running code

In [17]:
data = load_document("pdfs/guia_lgpd.pdf")
# print(data[1].page_content)

print(f"You have {len(data)} pages in your data")

Loading pdfs/guia_lgpd.pdf
You have 69 pages in your data


In [18]:
data = load_from_wikipedia("Gemini-AI")
print(data[0].page_content)

Gemini (formerly known as Bard) is a generative artificial intelligence chatbot and virtual assistant developed by Google. Based on the large language model (LLM) of the same name, it was launched in March 2023 in response to the rise of OpenAI's ChatGPT.


== History ==


=== Background ===

In November 2022, OpenAI launched ChatGPT, a chatbot based on the GPT-3 family of large language models (LLMs). ChatGPT gained worldwide attention, becoming a viral Internet sensation. Alarmed by ChatGPT's potential threat to Google Search, Google executives issued a "code red" alert, reassigning several teams to assist in the company's artificial intelligence (AI) efforts. Sundar Pichai, the CEO of Google and parent company Alphabet, was widely reported to have issued the alert, but Pichai later denied this to The New York Times. In a rare move, Google co-founders Larry Page and Sergey Brin, who had stepped down from their roles as co-CEOs of Alphabet in 2019, attended emergency meetings with com